#Tetillas_mosca
El código a continuación fue escrito por alumnos de INB, UNAM, para facilitar la cuantificación de la densidad de las tetillas presentes en el lente del omatidio de la mosca de la fruta (*Drosophila melanogaster*).
Favor de consultar el README en este repositorio para conocer los detalles relacionados. 

In [17]:
## Imports necesarios
import numpy as np
import pandas as pd

Cargar los datos de entrada

In [12]:
pezones = pd.read_csv('https://raw.githubusercontent.com/DanielT-M/tetillas_mosca/main/tablas/Pezones_control_coordenadas.csv') 

pezones #dataframe con lo datos de entrada

,x1,y1
0,0.154,0.239
1,0.335,0.098
2,0.314,0.402
3,0.095,0.473
4,0.270,0.616
...,...,...
382,5.390,0.453
383,5.241,0.268
384,5.023,0.227
385,5.232,0.089


In [14]:
#Definimos una función "tetillas" que organice los datos en una sola ejecución

def tetillas(data, r):

  datos = data.copy()
  #definimos listas

  lenght = []
  punto_1 = []
  punto_2 = []

  #Definimos el ciclo for:
  for i in range(0,len(datos)):  # i va a recorrer todos los números de 0 hasta la longitud del data frame
    for n in range(0,len(datos)): 
      #Agregamos a la lista "lenght" cada distancia obtenida
      lenght.append(np.sqrt((np.power((datos['x1'][n]-datos['x1'][i]),2)+(np.power((datos['y1'][n]-datos['y1'][i]),2) ))))
      #Tomamos el registro de entre cuales puntos se midió la distancia
      punto_1.append(i)
      punto_2.append(n)

  # Con las listas creamos un dataframe 

  listas = [punto_1,punto_2,lenght]

  df_pezones = pd.DataFrame (listas).transpose()
  df_pezones.columns = ['Punto 1', 'Punto 2', 'Distancia (µm)']

  #df_pezones contiene todas las distancias y los puntos que se comparan
  
  #Eliminamos las distancias mayores al radio (r)

  df_pezones.drop(df_pezones[(df_pezones['Distancia (µm)'] > r ) ].index, inplace=True)

  #En los datos de entrada queremos eliminar los puntos fuera del margen dado por r
  max_x = datos['x1'].max()
  max_y = datos['y1'].max()

  datos.drop(datos[(datos['x1'] > max_x-r) | (datos['x1'] < r) | (datos['y1'] < r) | (datos['y1'] > max_y-r)].index, inplace=True)

  #Creamos un arreglo con los puntos restantes identificados por numero
  arreglo_pezones = datos.index.values

  #De df_pezones nos quedamos con los casos donde el punto 1 esta dentro del margen
  # y eliminamos las distancias cero

  for i in df_pezones.index.values:
    if not ((df_pezones.loc[i]['Punto 1'] in arreglo_pezones) & (df_pezones.loc[i]['Distancia (µm)'] != 0)):
      df_pezones = df_pezones.drop(i)
  
  print(df_pezones)
  #Creamos una tabla de frecuencias para saber cuántos puntos rodean el punto 1
  #dadas nuestras condiciones
  
  frec = df_pezones.groupby(['Punto 1']).count()
  frec = frec.drop(columns = ['Distancia (µm)']) 
  frec = frec.rename(columns={"Punto 2": "Cantidad de puntos alrededor"})
  print(frec)
  
  #Exportamos los dataframes

  df_pezones.to_csv("Distancias.csv",index=False)
  frec.to_csv("Conteo.csv")
  datos.to_csv("Coordenadas_de_los_puntos.csv",index=True, index_label="No. de Punto")
  


Insertar el nombre de los datos de entrada y el radio deseado en *µ*m

In [15]:
tetillas(pezones, 0.75) #r = 0.75 µm

        Punto 1  Punto 2  Distancia (µm)
18965      49.0      2.0        0.683441
18967      49.0      4.0        0.631861
18968      49.0      5.0        0.712310
18969      49.0      6.0        0.462180
18970      49.0      7.0        0.433889
...         ...      ...             ...
142793    368.0    377.0        0.447385
142794    368.0    378.0        0.513640
142795    368.0    379.0        0.651685
142796    368.0    380.0        0.602416
142797    368.0    381.0        0.683878

[5527 rows x 3 columns]
         Cantidad de puntos alrededor
Punto 1                              
49.0                               33
50.0                               36
69.0                               36
70.0                               38
71.0                               37
...                               ...
348.0                              30
349.0                              30
350.0                              32
351.0                              30
368.0                      

In [ ]:
tetillas(pezones, 0.5) #r = 0.5 µm

        Punto 1  Punto 2  Distancia (µm)
4259       11.0      2.0        0.460184
4261       11.0      4.0        0.456089
4263       11.0      6.0        0.375868
4264       11.0      7.0        0.240052
4265       11.0      8.0        0.304867
...         ...      ...             ...
146632    378.0    346.0        0.249032
146633    378.0    347.0        0.450960
146662    378.0    376.0        0.430141
146663    378.0    377.0        0.212257
146665    378.0    379.0        0.419315

[3189 rows x 3 columns]
         Cantidad de puntos alrededor
Punto 1                              
11.0                               14
12.0                               15
13.0                               14
48.0                               14
49.0                               15
...                               ...
368.0                              15
369.0                              14
376.0                              15
377.0                              12
378.0                      